# Creating tear sheets with Alphalens

In the previous lesson, you learned what Alphalens is. In this lesson, you will learn a four step process for how to use it:
1. Express an alpha factor and define a trading universe by creating and `running a pipeline` over a certain time period.
2. Query pricing data for the assets in our universe during that same time period with `get_pricing()`
3. Align the alpha factor data with the pricing data with `get_clean_factor_and_forward_returns()`
4. Visualize how well our alpha factor predicts future price movements with `create_full_tear_sheet()`

## Build And Run A Pipeline
The following code expresses an alpha factor based on asset growth, then runs it with `run_pipeline()`

In [1]:
from zipline.pipeline.fundamentals import Fundamentals

from zipline.pipeline import Pipeline
from zipline.research import run_pipeline
from zipline.pipeline.builtin import QTradableStocksCN


def make_pipeline():
    
    # Measures a company's asset growth rate.
    # 改用报告期净资产增长率
    asset_growth = Fundamentals.financial_indicators.净资产增长率.latest 
    
    return Pipeline(
        columns = {'Asset Growth': asset_growth},
        screen = QTradableStocksCN() & asset_growth.notnull()
    )

pipeline_output = run_pipeline(pipeline=make_pipeline(), start_date='2014-1-1', end_date='2016-1-1')

# Show the first 5 rows of factor_data
pipeline_output.head(5) 

Asset Growth
datetime                  asset                     
2014-01-02 00:00:00+00:00 平安银行(000001)        11.927
                          万 科Ａ(000002)         6.957
                          国农科技(000004)         1.151
                          世纪星源(000005)        -4.252
                          深振业Ａ(000006)         8.778

## Query Pricing Data

Now that we have factor data, let's get pricing data for the same time period. `get_pricing()` returns pricing data for a list of assets over a specified time period. It requires four arguments:
- A list of assets for which we want pricing.
- A start date
- An end date
- Whether to use open, high, low or close pricing.

Execute the following cell to get pricing data.

In [2]:
from zipline.research import *

In [3]:
pricing_data = get_pricing(
    # Finds all assets that appear at least once in "factor_data"
    symbols_=pipeline_output.index.levels[1], # 使用参数`symbols_`避免与函数`symbols`混淆
    start_date='2014-1-1',
    end_date='2016-2-1',  # must be after run_pipeline()'s end date. Explained more in lesson 4
    fields='open'   # Generally, you should use open pricing.
)

# Show the first 5 rows of pricing_data
pricing_data.head(5)

,平安银行(000001),万 科Ａ(000002),国农科技(000004),世纪星源(000005),深振业Ａ(000006),全新好(000007),神州高铁(000008),中国宝安(000009),美丽生态(000010),深物业A(000011),...,醋化股份(603968),银龙股份(603969),金诚信(603979),中电电机(603988),艾华集团(603989),洛阳钼业(603993),*ST中新(603996),继峰股份(603997),方盛制药(603998),读者传媒(603999)
open,,,,,,,,,,,,,,,,,,,,,
2014-01-02 00:00:00+00:00,8.229,7.327,11.64,2.50,4.737,13.90,2.069,7.779,7.77,7.380,...,NaN,NaN,NaN,NaN,NaN,2.092,NaN,NaN,NaN,NaN
2014-01-03 00:00:00+00:00,8.250,7.309,11.70,2.49,4.650,14.04,2.064,7.821,8.08,7.437,...,NaN,NaN,NaN,NaN,NaN,2.041,NaN,NaN,NaN,NaN
2014-01-06 00:00:00+00:00,8.073,7.171,11.77,2.44,4.554,14.13,2.071,7.597,7.51,7.208,...,NaN,NaN,NaN,NaN,NaN,2.015,NaN,NaN,NaN,NaN
2014-01-07 00:00:00+00:00,7.829,6.814,12.00,2.39,4.276,14.04,1.956,7.472,7.13,6.961,...,NaN,NaN,NaN,NaN,NaN,1.934,NaN,NaN,NaN,NaN
2014-01-08 00:00:00+00:00,7.903,6.832,12.20,2.43,4.276,14.41,1.929,7.505,7.24,7.008,...,NaN,NaN,NaN,NaN,NaN,1.950,NaN,NaN,NaN,NaN


## Align Data

`get_clean_factor_and_forward_returns()` aligns the factor data created by `run_pipeline()` with the pricing data created by `get_pricing()`, and returns an object suitable for analysis with Alphalens' charting functions. It requires two arguments:
- The factor data we created with `run_pipeline()`.
- The pricing data we created with `get_pricing()`.

Execute the following cell to align the factor data with the pricing data.

In [4]:
from alphalens.utils import get_clean_factor_and_forward_returns

factor_data = get_clean_factor_and_forward_returns(
    factor=pipeline_output, 
    prices=pricing_data
)

# Show the first 5 rows of merged_data
factor_data.head(5) 

Dropped 0.0% entries from factor data: 0.0% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!


1D        5D       10D  factor  \
date                      asset                                                
2014-01-02 00:00:00+00:00 平安银行(000001)  0.002552 -0.035484 -0.037915  11.927   
                          万 科Ａ(000002) -0.002457 -0.070015 -0.101406   6.957   
                          国农科技(000004)  0.005155  0.051546 -0.002577   1.151   
                          世纪星源(000005) -0.004000 -0.040000 -0.068000  -4.252   
                          深振业Ａ(000006) -0.018366 -0.107452 -0.127718   8.778   

                                        factor_quantile  
date                      asset                          
2014-01-02 00:00:00+00:00 平安银行(000001)                5  
                          万 科Ａ(000002)                4  
                          国农科技(000004)                2  
                          世纪星源(000005)                1  
                          深振业Ａ(000006)                5

## Visualize Results

Finally, execute the following cell to pass the output of `get_clean_factor_and_forward_returns()` to a function called `create_full_tear_sheet()`. This will create whats known as a tear sheet.

In [5]:
from alphalens.tears import create_full_tear_sheet

create_full_tear_sheet(factor_data)

## That's It!

In the next lesson, we will show you how to interpret the charts produced by `create_full_tear_sheet()`.